# IMPORT MODULES

*First, import some important Python libraries and methods that you will need to process financial data and perform data analysis. Includes the freely available bug fix for the Yahoo Finance Pandas datareader.*

*Numpy is a Python library used for numerical computations using multidimensional arrays and complex mathematical functions.*

*Pandas is a Python library that is built from the ground-up to do financial data analysis. It has a dataframe object that makes it easy to analyze tabular data traditionally done using spreadsheets.*

*Matplotlib is a Python library used for visualizing data. Pandas provides a wrapper to the library so you can plot nice charts with a single line of code.*

In [ ]:
import numpy as np
import pandas as pd
import pandas_datareader.data as pdr
import yfinance as yf
#import fix_yahoo_finance as yf
yf.pdr_override()
import matplotlib.pyplot as plt
plt.style.use('seaborn')
from datetime import datetime

# PROCESS DATA

*While the API of Yahoo Finance is not stable, it gives you free access to historical end of day data of a broad range of global assets for long timeframes without requiring much processing*. 

*Note that we need to use adjusted closing prices instead of closing prices to create an equity time series. Corporate actions, such as dividend payouts and stock splits, distort prices which need to be adjusted to give you the correct closing prices for each day.*

*If the code below gives you an error, just wait a minute and try again. As mentioned earlier, the API is unstable. Review the 'Processing Data' workbook to refresh your memory about some of the methods involved in doing financial data processing and analysis.*

---



In [ ]:
start = datetime(2016, 9, 1)
end = datetime(2019, 8, 31)

stock = pdr.get_data_yahoo('MSFT', start, end)
stock.describe()

# EXPLORE SIGNALS

*A moving average is a based on a lookback window of a fixed time period (such as 20 days) that keeps moving in time. The idea is to detect a trend in the security by smoothing out the daily price fluctuations and capturing it in the moving average.* 

*In a dual moving average crossover algorithm, you compare a shorter term moving average, such as a 20 day simple moving average (SMA), with a longer term moving average, such as a 50 day SMA, to detect price momentum and generate the dual moving average crossover signal.*

*The value of the signal is +1 when the 20-day SMA is above the the 50-day SMA and -1 when the opposite occurs. Use the rolling.mean() method of pandas to calculate the simple moving averages.*

---



In [ ]:
SMA1 = 20
SMA2 = 50
stock['SMA1'] = stock['Adj Close'].rolling(SMA1).mean()
stock['SMA2'] = stock['Adj Close'].rolling(SMA2).mean()
stock.dropna(inplace=True)
stock[['Adj Close','SMA1','SMA2']].plot(figsize=(20,5), grid = True, title='The 20 and 50 day simple moving averages'), plt.show();

*It is important to separate the analysis of a signal from the analysis of a profitable trading strategy. A high quality signal can be used in different trading strategies. We create a new dataframe to capture the relevant data to analyze the dual moving average crossover signal.*  

*The 'change' column measures the absolute change in the closing prices of the stock using the diff() method. The 'crossover' column measures the absolute difference between the two moving averages at the end of each day. We use the Numpy where() conditional method to generate the dual moving average crossover signal.* 

---

In [ ]:
dualsma = pd.DataFrame()
dualsma['change'] = stock['Adj Close'].diff()
dualsma['crossover'] = stock['SMA1'] - stock['SMA2']
dualsma['signal']= np.where(dualsma['crossover'] > 0, 1, -1)
dualsma.sample(10)

# BACKTEST STRATEGY

*Backtesting enables you to simulate the past performance of a trading strategy using historical data. If done correctly, it can provide valuable information in assessing the future profitability of that trading strategy. The code below is based on 'Python For Finance' by Y. Hilpisch.*

*A new 'position' column is created using the where() method of Numpy. When the dual moving average crossover signal is positive, the strategy goes long (buys) the stock. When the opposite occurs, you can either go short (borrow and sell the stock) or go flat (sell the stock you bought earlier). A position value of -1 signifies a short position and 0 signifies a flat position.*

*A new 'returns' column is created for calculating the continuously compounded daily return of the stock using the log() method. To calculate the continuously compounded daily return, we take the natural logarithm of the simple gross daily return. The advantage of using logarithms to calculate continuously compounded returns is that you can merely add the terms in the column instead of multiplying them to get the continuoulsy compounded returns over any desired holding period. By converting multiplication into addition in log space, the computational complexity of the vectorization is reduced substantially.*


*It is also important to make sure that you shift the date one day into the past when you calculate the returns from the dual crossover strategy. This is because the position (long, short or flat) is decided the day before the return is realized. Otherwise you would include a look-ahead bias into your backtesting.*

---



In [ ]:
stock['position'] = np.where(stock['SMA1'] > stock['SMA2'], 1, 0)
stock['returns'] = np.log(stock['Adj Close']/stock['Adj Close'].shift(1))
stock['strategy'] = stock['position'].shift(1) * stock['returns']
stock.dropna(inplace=True)
stock[['returns', 'strategy']].plot(figsize=(20,5), grid = True, title='Dual cross-over strategy versus long only returns'), plt.show();

# EVALUATE PERFORMANCE

*We compare the performance of the dual moving average strategy with a long only strategy to see if we are able to take advantage of our signal.* 

*We use the exp() method or exponential function to convert the log returns back into the simple gross returns of both types of strategies.* 

*Note that this type of backtesting does not take into account bid-ask spread costs, commissions and liquidity related issues of real-life trading which would reduce the simple gross returns of both strategies.* 

*Vectorized backtesting is a simple first step. Only if the strategy performs well in this step, does it makes sense to spend time backtesting the strategy using more advanced backtesting code or libraries.*

In [ ]:
np.exp(stock[['returns', 'strategy']].sum())

In [ ]:
stock[['returns', 'strategy']].describe()

In [ ]:
plt.hist(stock.returns), plt.title('Long only returns'), plt.xlabel('High-Low Ranges'), plt.ylabel('Frequency'), plt.show();
print(stock.returns.describe());

In [ ]:
plt.hist(stock.strategy), plt.title('SMA Crossover Strategy'), plt.xlabel('High-Low Ranges'), plt.ylabel('Frequency'), plt.show();
print(stock.strategy.describe());